In [ ]:
#This cell code seperated the images that are mentioned in the csv file
import csv
import os
import numpy as np
import shutil
import matplotlib.pyplot as plt

#loading the given days data
with open('day_numbers.csv',newline='',mode='r') as csvfile:
    csvreader=csv.reader(csvfile)
    for row in csvreader:
        days=row
#loadin all the images       
figs=[]  
for r,d,f in os.walk('figures/observed speeds'):
      for file in f:
            if '.png' in file:
                  figs.append(os.path.join(r,file))
#loading the images that are mentioned in the day numbers
selected_figs=[]
for number in days:
    selected_figs.append(figs[int(number)-1])
    
#Seperating the images that are mentioned in the day numbers
for num,fig in enumerate(selected_figs):
    shutil.copy(fig,'Speed_figs_ideal/{}.png'.format(num))
    
# Drawing contour plots from the speed data  
TS = np.loadtxt("TS_speed.csv", delimiter=",")
X, Y = (TS[:,0], TS[:,1])
#TS.shape[1]
for n in range(2,TS.shape[1]):
    Z=TS[:,n]
    xs=[]
    for k in X:
        if k not in xs:
            xs.append(k)
            
    ys=[]
    for j in Y:
        if j not in ys:
            ys.append(j)
    Z=Z.reshape(len(xs),len(ys))
    Z=np.transpose(Z)
    plt.figure()
    plt.contourf(xs,ys,Z,10)
    plt.gca().axes.get_yaxis().set_visible(False)
    plt.gca().axes.get_xaxis().set_visible(False)
    #plt.colorbar()
    plt.savefig('Speed_figs/{}.png'.format(n-2))
    
    
# Drawing contour plots from the flow data  
TS = np.loadtxt("TS_flow.csv", delimiter=",")
X, Y = (TS[:,0], TS[:,1])
#TS.shape[1]
for n in range(2,TS.shape[1]):
    Z=TS[:,n]
    xs=[]
    for k in X:
        if k not in xs:
            xs.append(k)
            
    ys=[]
    for j in Y:
        if j not in ys:
            ys.append(j)
    Z=Z.reshape(len(xs),len(ys))
    Z=np.transpose(Z)
    plt.figure()
    plt.contourf(xs,ys,Z,10)
    plt.gca().axes.get_yaxis().set_visible(False)
    plt.gca().axes.get_xaxis().set_visible(False)
    #plt.colorbar()
    plt.savefig('flow_figs/{}.png'.format(n-2))

In [ ]:
#This cell code prepares the data
import os
import csv
from scipy import ndimage
import numpy as np
import matplotlib.pyplot as plt
import cv2
import random
import shutil
"""
img=cv2.imread('Speed_figs_ideal/52.png')
x=(len(img[:,0,0]))
y=(len(img[0,:,0]))
img1=img[int(0.07*x):x-int(0.04*x),int(0.08*y):x-int(0.16*y),:]
#img1=img
hsv = cv2.cvtColor(img1,cv2.COLOR_BGR2HSV)

#lower red
lower_red = np.array([0,50,50])
upper_red = np.array([10,255,255])


#upper red
lower_red2 = np.array([170,50,50])
upper_red2 = np.array([180,255,255])

mask = cv2.inRange(hsv, lower_red, upper_red)
res = cv2.bitwise_and(img1,img1, mask= mask)

print('red',np.count_nonzero(mask))
print('white',np.count_nonzero(res))

cv2.imshow('mask',mask)
cv2.imshow('res',res)
cv2.waitKey(0)
cv2.destroyAllWindows()
"""
#extracting the pattern of each speed figure
tr_spots=[] 
tr_dict={} 
for r,d,f in os.walk('Speed_figs_ideal'):
      for file in f:
            if '.png' in file:         
                img=cv2.imread(os.path.join(r,file))
                x=(len(img[:,0,0]))
                y=(len(img[0,:,0]))
                img1=img[int(0.07*x):x-int(0.04*x),int(0.08*y):x-int(0.16*y),:]
                hsv = cv2.cvtColor(img1,cv2.COLOR_BGR2HSV)
                lower_red = np.array([0,50,50])
                upper_red = np.array([10,255,255])
                mask = cv2.inRange(hsv, lower_red, upper_red)
                res = cv2.bitwise_and(img1,img1, mask= mask)
                tr_spots.append(np.count_nonzero(res))
                tr_dict[file]=np.count_nonzero(res)       
#calculating the value of traffic in every image
max_tr=max(tr_spots)
min_tr=min(tr_spots)
mid_tr=np.average(tr_spots)

#creating 5 classes with different values of traffic
for i in range(1,6):
    globals()['Class{}'.format(i)]=[]
seperator=max_tr/5    
for key in tr_dict.keys():
    if tr_dict[key]<seperator:
        Class1.append(key)
    elif tr_dict[key]>=seperator and tr_dict[key]<2*seperator:
        Class2.append(key)
    elif tr_dict[key]>=2*seperator and tr_dict[key]<3*seperator:
        Class3.append(key)
    elif tr_dict[key]>=3*seperator and tr_dict[key]<4*seperator:
        Class4.append(key)    
    elif tr_dict[key]>=4*seperator and tr_dict[key]<=5*seperator:
        Class5.append(key)
        
#Correcting 8 images in class1 that belong to class2       
replace=[11,39,65,80,81,98,101,111]
for re in replace:
    Class1.remove(str(re)+'.png')
    Class2.append(str(re)+'.png')

#moving each class images to the righ folder
for g in range(1,6):
    try:
        os.mkdir('classes/Class{}/'.format(g))
    except:
        pass
    for f in globals()['Class{}'.format(g)].copy():
        img=cv2.imread('Speed_figs_ideal/'+f)
        cv2.imwrite('classes/Class{}/'.format(g)+f,img)
      
        
        
# Creating more data from images by rotating 180 degree the images of the class 2,3,4,5    
thetas=[90,180,270]
for c in Class5.copy():
    img=cv2.imread('flow_figs/'+c)
    theta1=thetas[1]
    #theta2=thetas[1]
    #theta3=thetas[2]
    rotated_img1=ndimage.rotate(img,theta1)
    #rotated_img2=ndimage.rotate(img,theta2)
    #rotated_img3=ndimage.rotate(img,theta3)
    Class5.append(c[:-4]+'_1.png')
    #Class5.append(c[:-4]+'_2.png')
    #Class5.append(c[:-4]+'_3.png')
    cv2.imwrite('flow_figs/'+c[:-4]+'_1.png',rotated_img1)
    #cv2.imwrite('flow_figs/'+c[:-4]+'_2.png',rotated_img2)
    #cv2.imwrite('flow_figs/'+c[:-4]+'_3.png',rotated_img3)
for c in Class4.copy():
    img=cv2.imread('flow_figs/'+c)
    theta1=thetas[1]
    #theta2=thetas[1]
    rotated_img1=ndimage.rotate(img,theta1)
   # rotated_img2=ndimage.rotate(img,theta2)
    Class4.append(c[:-4]+'_1.png')
    #Class4.append(c[:-4]+'_2.png')
    cv2.imwrite('flow_figs/'+c[:-4]+'_1.png',rotated_img1)
    #cv2.imwrite('flow_figs/'+c[:-4]+'_2.png',rotated_img2)
for c in Class3.copy():
    img=cv2.imread('flow_figs/'+c)
    theta1=thetas[1]
    #theta2=thetas[1]
    rotated_img1=ndimage.rotate(img,theta1)
    #rotated_img2=ndimage.rotate(img,theta2)
    Class3.append(c[:-4]+'_1.png')
    #Class3.append(c[:-4]+'_2.png')
    cv2.imwrite('flow_figs/'+c[:-4]+'_1.png',rotated_img1)
    #cv2.imwrite('flow_figs/'+c[:-4]+'_2.png',rotated_img2)
for c in Class2.copy():
    img=cv2.imread('flow_figs/'+c)
    theta1=thetas[1]
    #theta2=thetas[1]
    rotated_img1=ndimage.rotate(img,theta1)
    #rotated_img2=ndimage.rotate(img,theta2)
    Class2.append(c[:-4]+'_1.png')
    #Class2.append(c[:-4]+'_2.png')
    cv2.imwrite('flow_figs/'+c[:-4]+'_1.png',rotated_img1)
    #cv2.imwrite('flow_figs/'+c[:-4]+'_2.png',rotated_img2)



#moving all the data of all classes into one dictionary

label_dic={}        
for i in range(1,6):
    for data in globals()['Class{}'.format(i)]:
        label_dic[data]=i
   
"""
label_dic['85.png']=2
label_dic['112.png']=2
label_dic['97.png']=2
label_dic['99.png']=2
label_dic['100.png']=2
label_dic['40.png']=2
"""

#saving the dictionary
with open("labels.csv", mode="w",newline='') as csvfile :
    csvwriter=csv.writer(csvfile)
    for key, val in label_dic.items():
        csvwriter.writerow([key,val])

#extracting and saving the patterns of each images in the righ class
for il in range(1,6):
    for im in globals()['Class{}'.format(il)]:
        try:
            if im[-6]=='_':
                continue
        except:
            pass
        img=cv2.imread(os.path.join('classes','Class{}'.format(il),im))
        x=(len(img[:,0,0]))
        y=(len(img[0,:,0]))
        img1=img[int(0.07*x):x-int(0.04*x),int(0.08*y):x-int(0.16*y),:]
        hsv = cv2.cvtColor(img1,cv2.COLOR_BGR2HSV)
        lower_red = np.array([0,50,50])
        upper_red = np.array([10,255,255])
        mask = cv2.inRange(hsv, lower_red, upper_red)
        try:
            os.mkdir(os.path.join('classes','Class{}'.format(il),'masks'))
        except:
            pass
        cv2.imwrite('classes/'+'Class{}/'.format(il)+'masks/'+im,mask)
     
    
 # creating vlaidation data
for m in range(1,6):
    validation=[]
    for n in range(len(globals()['Class{}'.format(m)])//5):
        validation.append(globals()['Class{}'.format(m)][random.randint(0,len(globals()['Class{}'.format(m)])-1)])
    for v in validation:
        shutil.copy('flow_figs/'+v ,'Validation')
    

In [ ]:
#This cell code creates a sample pattern from the patterns of each class
import os
import numpy as np
import cv2
#readin all the patterns of a class and getting the average of them
for i in range(2,6):
    path='classes/Class{}/masks'.format(i)
    full_image=[]
    for r,d,f in os.walk(path):
      for file in f:
          if 'pattern.png' in file:
              continue
          if '.png' in file:  
                img=cv2.imread(os.path.join(r,file))[:,:,0]
                full_image.append(img)
    avg_img=np.zeros((full_image[0].shape[0],full_image[0].shape[1]),dtype=np.float64)
    for fi in range(len(full_image)):
        avg_img=avg_img+full_image[fi]
    avg_img=avg_img/len(full_image)
    """        
    avg_value=0
    for i in avg_img.copy():
        for j in i:
            avg_value+=j
    avg_value/=avg_img.shape[0]*avg_img.shape[1]
    """
    max_value=avg_img.max()            
     
    avg_img2=avg_img.copy() 
    #filtering the averaged pattern by some selected value
    for n,i in enumerate(avg_img.copy()):
        for m,j in enumerate(i):
            if j<max_value/2.5: # The value of 2.5 has been selected due to the programmer observations
                avg_img2[n][m]=0
    avg_img2=avg_img2.astype(np.uint8)
            #else:
                #avg_img2[n][m]=255
    # adding x and y axis to the sample pattern
    final_img=np.zeros((922,873,3),dtype=np.uint8)                
    sample_img=cv2.imread('Speed_figs_ideal/0.png')
    x=(len(sample_img[:,0,0]))
    y=(len(sample_img[0,:,0]))
    #sample_img=cv2.cvtColor(sample_img,cv2.COLOR_BGR2GRAY)
    final_img[0:int(0.07*x),:,:]=sample_img[0:int(0.07*x),:,:].copy()
    final_img[x-int(0.04*x):,:,:]=sample_img[x-int(0.04*x):,:,:].copy()
    final_img[:,0:int(0.08*y),:]=sample_img[:,0:int(0.08*y),:].copy()
    final_img[:,x-int(0.16*y):,:]=sample_img[:,x-int(0.16*y):,:].copy()
    
    final_img[int(0.07*x):x-int(0.04*x),int(0.08*y):x-int(0.16*y),2]=avg_img2.copy()
    #final_img[int(0.07*x):x-int(0.04*x),int(0.08*y):x-int(0.16*y),0]=avg_img3.copy()
    #final_img[int(0.07*x):x-int(0.04*x),int(0.08*y):x-int(0.16*y),1]=avg_img2.copy()
    #final_img[int(0.07*x):x-int(0.04*x),int(0.08*y):x-int(0.16*y),2]=avg_img2.copy()
    final_img=final_img[int(0.07*x):,0:x-int(0.16*y),:]
    cv2.imwrite(path+'/pattern.png',final_img) 
#cv2.imshow('gg',final_img)
#cv2.waitKey(0)